# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e:
    print(e)
    

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_library"
query = query + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"

"""Our choice for (sessionId, itemInSession) as our PRIMARY KEY is because 
we'll be using those columns to filter our query."""

session.execute(query)


                    

In [9]:
# transformation and loading part of the ETL
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_library (artist, song, length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
query = "SELECT * FROM songs_library WHERE sessionId=338 AND itemInSession=4;"
rows = session.execute(query)
df = pd.DataFrame(list(rows))
df

,sessionid,iteminsession,artist,length,song
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
query2 = "CREATE TABLE IF NOT EXISTS user_player"
query2 = query2 + "(artist text, song text, firstName text, lastName text, itemInSession int, userId int, sessionId int, \
PRIMARY KEY ((userId, sessionId), itemInSession))"

"""We need to choose ((userId, sessionId), ItemInSession) as PRIMARY KEY
because we'll identify the rows using (userId, sessionId) and sort using ItemInSession as our Clustering column."""

session.execute(query2)

                    

In [14]:
# transformation and loading part of the ETL
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_player (userId, sessionId, artist, song, firstName, lastName, ItemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

In [15]:
query = "SELECT * FROM user_player WHERE userId=10 AND sessionId=182;"
rows = session.execute(query)
df = pd.DataFrame(list(rows))
df

,userid,sessionid,iteminsession,artist,firstname,lastname,song
0,10,182,0,Down To The Bone,Sylvie,Cruz,Keep On Keepin' On
1,10,182,1,Three Drives,Sylvie,Cruz,Greece 2000
2,10,182,2,Sebastien Tellier,Sylvie,Cruz,Kilometer
3,10,182,3,Lonnie Gordon,Sylvie,Cruz,Catch You Baby (Steve Pitron & Max Sanna Radio...


In [16]:
query3 = "CREATE TABLE IF NOT EXISTS user_info "
query3 = query3 + "(userId int, firstName text, lastName text, song text, PRIMARY KEY (song, userId))"

"""Here we simple chose (song, userId) as our PRIMARY KEY because we filter our query using song and 
we use userId to identify unique names."""
session.execute(query3)
                    

In [17]:
# transformation and loading part of the ETL
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header

    for line in csvreader:

        query = "INSERT INTO user_info (userId, firstName, lastName, song)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

In [18]:
query = "SELECT userId, firstName, lastName FROM user_info WHERE song='All Hands Against His Own';"
rows = session.execute(query)
df = pd.DataFrame(list(rows))
df

,userid,firstname,lastname
0,29,Jacqueline,Lynch
1,80,Tegan,Levine
2,95,Sara,Johnson


### Drop the tables before closing out the sessions

In [19]:
query = "drop table songs_library"
session.execute(query)
query2 = "drop table user_player"
session.execute(query2)
query3 = "drop table user_info"
session.execute(query3)



### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()